In [109]:
import re
import pandas as pd
from profanity import profanity
import csv
import numpy as np
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import nltk
import spacy
from textblob import TextBlob
nlp = spacy.load('de_core_news_sm')
RAW_DATA_PATH = 'data/raw/'

Helper Functions

In [110]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [111]:
def find_longest_word(word_list):
    longest_word =  max(word_list, key=len)
    return len(longest_word)

In [168]:
def lemmatize_string(input_text):
    doc = nlp(input_text.lower())
    result = ' '.join([x.lemma_ for x in doc])
    doc = nlp(result.title())
    result = ' '.join([x.lemma_ for x in doc]).upper()
    return result

Data Extraction

In [112]:
df_Lied = pd.read_csv(RAW_DATA_PATH + 'LIED.csv', usecols=['ID','INTERPRET', 'TITEL', 'SPRACHE_DEUTSCH', 'TEXT_TEIL1', 'TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4'])
print(df_Lied.head())

df_Chart_Position = pd.read_csv(RAW_DATA_PATH + 'CHART_POSITION.csv', usecols=['LIED_ID', 'POSITION', 'DATUM_VON', 'DATUM_BIS'])
print(df_Chart_Position.head())

with open(RAW_DATA_PATH+'Stoppwords.csv', newline='', encoding='UTF-8') as f:
    stopwords_list = list(csv.reader(f))
stopwords_list = [word.upper() for word in flatten(stopwords_list)]

        ID INTERPRET    TITEL SPRACHE_DEUTSCH  \
0  1148163     Yasha  Strand               Ja   
1  1148163     Yasha  Strand               Ja   
2  1148163     Yasha  Strand               Ja   
3  1148163     Yasha  Strand               Ja   
4  1148163     Yasha  Strand               Ja   

                                          TEXT_TEIL1 TEXT_TEIL2 TEXT_TEIL3  \
0  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...        NaN        NaN   
1  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...        NaN        NaN   
2  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...        NaN        NaN   
3  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...        NaN        NaN   
4  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...        NaN        NaN   

  TEXT_TEIL4  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  
   LIED_ID  POSITION   DATUM_VON   DATUM_BIS
0    26024        26  1983-04-18  1983-04-24
1     1073        46  1984-06-04  1984-06-10
2     1131 

Data Conversion

In [113]:
df_Lied['Text'] = df_Lied['TEXT_TEIL1'].fillna('') + df_Lied['TEXT_TEIL2'].fillna('') + df_Lied['TEXT_TEIL3'].fillna('') + df_Lied['TEXT_TEIL4'].fillna('')
#df_Lied.drop(columns=['TEXT_TEIL1', 'TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4'], axis=1, inplace=True)

df_Chart_Position['DATUM_VON'] = pd.to_datetime(df_Chart_Position['DATUM_VON'])
df_Chart_Position['DATUM_BIS'] = pd.to_datetime(df_Chart_Position['DATUM_BIS'])
df_Chart_Position['DAUER'] = (df_Chart_Position['DATUM_BIS'] - df_Chart_Position['DATUM_VON']).dt.days.astype('int16')
df_Chart_Position['Jahr'] = df_Chart_Position['DATUM_BIS'].dt.year.astype('int16')
df_Chart_Position['Monat'] =  df_Chart_Position['DATUM_BIS'].dt.month.astype('int16')

Text Preprocessing

In [188]:
df_Lied['processed_Text'] = df_Lied['Text']

#lemmatization
df_Lied['processed_Text'] = df_Lied.processed_Text.apply(lambda text: lemmatize_string(text))

#Stemming???
#df_Lied['processed_Text'] = df_Lied.processed_Text.apply(lambda text: lemmatize_string(text))

# tokenize
df_Lied['processed_Text'] = df_Lied.processed_Text.apply(lambda text: nltk.word_tokenize(text))

#remove stopwords
df_Lied['processed_Text'] = df_Lied['processed_Text'].apply(lambda x: [item for item in x if item not in stopwords_list])

#remove numbers
df_Lied['processed_Text'] = df_Lied['processed_Text'].apply(lambda word_list : [re.sub('\w*\d\w*','', word) for word in word_list])

# ngram


0    [LASS, STRAND, WEISST, BRAUCH, MAL, WIEDER, ME...
1    [LASS, STRAND, WEISST, BRAUCH, MAL, WIEDER, ME...
2    [LASS, STRAND, WEISST, BRAUCH, MAL, WIEDER, ME...
3    [LASS, STRAND, WEISST, BRAUCH, MAL, WIEDER, ME...
4    [LASS, STRAND, WEISST, BRAUCH, MAL, WIEDER, ME...
Name: processed_Text, dtype: object


In [189]:
print(df_Lied['processed_Text'][0])

['LASS', 'STRAND', 'WEISST', 'BRAUCH', 'MAL', 'WIEDER', 'MEER', 'SAND', 'YEAH', 'BRAUCH', 'MAL', 'WIEDERGRUND', 'BLEIBEN', 'AI', 'UHR', 'FAELLT', 'BLAU', 'SEHEN', 'VERSINKT', 'SETZTEN', 'LANGSAM', 'WASSER', 'SPIEGELGLATT', 'BLINKT', 'WOLLEN', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', 'IS', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', 'IS', 'STEHST', 'ZEIGST', 'ECHT', 'LICHTER', 'STADT', 'VERBRANNT', 'LASS', 'STRAND', 'KOMM', 'STEIGEN', 'LEUCHTEST', 'DIAMANT', 'YEHE', 'ALL', 'SALZ', 'HAUT', 'AI', 'SEH', 'GESTOCHEN', 'SCHARF', 'TUT', 'FAST', 'AUGEN', 'WEH', 'ENDLICH', 'NOCH', 'NIE', 'GESEHEN', 'YEHI', 'WOLLEN', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', 'IS', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', 'IS', 'STEHST', 'ZEIGST', 'ECHT', 'LICHTER', 'STADT', 'VERBRANNT', 'LASS', 'STRAND', 'LASS', 'STRAND', 'WEISST', 'LASS', 'STRAND', 'YEAH', 'BRAUCHGRUND', 'BLEIBEN', 'AI', 'LASS', 'STRAND', 'WEISST', 'BRAUCH', 'MAL', 'WIEDER', 'MEER', 'SAND', 'YEAH', 'BRAUCH', 'MAL', 'WIEDERGRUND', 'BLEIBEN',

In [190]:
print(df_Lied['Text'][0])

LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN ICH BRAUCH MAL WIEDER MEER UND SAND  YEAH BRAUCH MAL WIEDERGRUND ZU BLEIBEN  AI DEINE UHR FAELLT INS BLAU WIR SEHEN  WIE DIE ZEIT VERSINKT WIR SETZTEN LANGSAM AUF DAS WASSER SPIEGELGLATT UND BLINKT  WIR WOLLEN DIE SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS DU STEHST NEBEN MIR  ZEIGST MIR  DASS ES ECHT IST DIE LICHTER MEINER STADT SIND VERBRANNT LASS UNS ZUM STRAND    KOMM WIR STEIGEN AUS DU LEUCHTEST WIE AUS DIAMANT  YEHE MIT ALL DEM SALZ AUF DER HAUT  AI ICH SEH DICH GESTOCHEN SCHARF ES TUT FAST DEN AUGEN WEH UND ENDLICH BIST DU DA SO HABE ICH DICH NOCH NIE GESEHEN  YEHI  WIR WOLLEN DIE SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS DU STEHST NEBEN MIR  ZEIGST MIR  DASS ES ECHT IST DIE LICHTER MEINER STADT SIND VERBRANNT LASS UNS ZUM STRAND   LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN LASS UNS ZUM STRAND  YEAH ICH BRAUCHGRUND ZU BLEIB

In [79]:
print(df_Lied['tokenized_text'][1000])

KeyError: 'tokenized_text'

In [ ]:
#df_Lied['TEXT_LAENGE'] = list(len(word) for word in df_Lied['Text'])
df_Lied['ANZ_UNIQUE_WOERTER'] = list(len(set(word.split(" "))) for word in df_Lied['Text'])
df_Lied['LAENGE_LAENGSTES_WORT'] = list(len(max(set(word.split(" ")), key=len)) for word in df_Lied['Text'])
df_Lied[['LAENGE_LAENGSTES_WORT','ID']].to_csv('Data/processed/Wort_LAENGE_v2.csv')
df_Lied['INTERPRET'].value_counts(ascending=True, dropna=True, normalize=True)\
    #.to_csv('Data/processed/Interpret_Occurrences_normalised.csv')

MAX_RANK = 50
df_Chart_Position['RANK_SCORE'] = MAX_RANK - df_Chart_Position['POSITION'] + 1

Selection

In [115]:
df_Lied.drop(['TEXT_TEIL1','TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4'], axis=1, inplace=True)
df_Lied.drop(['SPRACHE_DEUTSCH'], axis=1, inplace=True)
df_Chart_Position.rename(columns={'ID':'Lied_ID'}, inplace=True)
df = df_Lied.merge(df_Chart_Position.rename(columns={'LIED_ID':'ID'}),on='ID', how='inner')
print(df.head())
print(df_Lied.columns)
print(df_Chart_Position.columns)

        ID INTERPRET    TITEL  \
0  1148163     Yasha  Strand    
1  1148163     Yasha  Strand    
2  1148163     Yasha  Strand    
3  1148163     Yasha  Strand    
4  1148163     Yasha  Strand    

                                                Text  ANZ_UNIQUE_WOERTER  \
0  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...                  92   
1  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...                  92   
2  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...                  92   
3  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...                  92   
4  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...                  92   

   LAENGE_LAENGSTES_WORT  POSITION  DATUM_VON  DATUM_BIS  DAUER  Jahr  Monat  \
0                     12        47 2013-08-23 2013-08-29      6  2013      8   
1                     12        19 2013-06-14 2013-06-20      6  2013      6   
2                     12        33 2013-07-05 2013-07-11      6  2013      7   
3                     12

profanity check

In [116]:
with open(RAW_DATA_PATH+'german-profanity-list.csv', newline='', encoding='UTF-8') as f:
    profanity_list = list(csv.reader(f))

profanity.load_words(flatten(profanity_list))

Percentage of Stopwords

In [117]:
#df['Number_of_Stopwords'] = df['Text'].str.split().apply(lambda x: len(set(x) & set(stopwords_list)))
#df['Stopword_Percentage'] = df.Number_of_Stopwords.apply(lambda row: row/len(df['Text']))
df_Lied[['ID','Number_of_Stopwords','Stopword_Percentage', 'RANK_SCORE']].to_csv('Data/processed/Stoppwords-RANK_SCORE.csv')

Sentiment Analysis

In [118]:
#df_Lied['SUBJEKTIVITY'] = df_Lied.processed_text.apply(lambda text: TextBlob(str(text)).sentiment.subjectivity)
df_Lied['POLARITY'] = df_Lied.processed_text.apply(lambda text: TextBlob(str(text)).sentiment.polarity)

AttributeError: 'DataFrame' object has no attribute 'processed_text'

In [ ]:
df_Lied[['ID','Number_of_Stopwords','Stopword_Percentage', 'RANK_SCORE']].to_csv('Data/processed/Sentiment.csv')

LDA (BoW) - Title

Sentiment(polarity=0.2, subjectivity=0.6)


Named Entity Recognition

In [32]:
test = [word for text in df_Lied['Text'] for word in nlp(text).ents]
print(test[0])

KeyboardInterrupt: 

Readability analysis

Title Analysis

In [240]:
titel_list = [word.upper() for word in list(set(df_Lied['TITEL']))]
tokenizer = RegexpTokenizer(r'\w+')
tokenized_list = [word for titel in titel_list for word in tokenizer.tokenize(titel)]
no_stopwords_title_list = [item for item in tokenized_list if item not in stopwords_list]
df_title_word_occurrence = pd.value_counts(np.array(no_stopwords_title_list))
df_title_word_occurrence.to_csv('Data/processed/title_word_occurrence.csv')

Seasonal determination

Boosted Words -Analysis

Merge df

In [ ]:
df = pd.merge(df_Lied, df_Chart_Position.set_axis(['DATUM_VON','DATUM_BIS', 'ID', 'POSITION', 'DAUER', 'BELIEBTHEIT', 'Jahr', 'Monat'], axis=1, inplace=False), on='ID', how='inner').sort_values(by=['ID'])
#print(df_Lied.isnull().values.any())#df muss noch ausgetauscht werden

Label Smoothing

Normalising

EDA